In [11]:
# packages
import pandas as pd
from bs4 import BeautifulSoup
import requests
from zipfile import ZipFile
import requests
import re
from datetime import date
import json
import xmltodict
import xml.etree.ElementTree as ET
import numpy as np

In [33]:
# get links from website

URL = "https://ds.marktstammdatenregister.dev/Marktstammdatenregister"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
DataLink = soup.findAll('a', href = re.compile('^https://download.marktstammdatenregister.de/Gesamtdatenexport_'))[0]['href']

!!! Download takes ~15 minutes !!!

In [ ]:
# download data
req = requests.get(DataLink)

In [6]:
# set filename
datum = date.today()
filename = f'MaStR_Gesamtdatenexport_{datum}.zip'


In [ ]:
# writing file to system
print('Download started')
with open(filename, 'wb') as output_file:
    output_file.write(req.content)
print('Download completed')

In [7]:
for name in filename.namelist():
    print ('%s' % (name))

filename.close()

AttributeError: 'str' object has no attribute 'namelist'

In [56]:
mastr_directory = "/home/jan/Uni/DS-Project/data/MaStR/"

# unzip folder and extract required data
Matches = []
with ZipFile("/home/jan/Uni/DS-Project/modules/data/MaStR_Gesamtdatenexport_2022-11-06", 'r') as ZipObject:
    for names in ZipObject.namelist():
        PVA = re.findall(r'^EinheitenSolar.*xml$', names)
        Matches.append(PVA)
        WKA = re.findall(r'^EinheitenWind.*xml$', names)
        Matches.append(WKA)
    while [] in Matches :
        Matches.remove([])
    Matches = [str(M) for M in Matches]
    for i in range(len(Matches)-1):
        Source_Name = Matches[i][2:-2]
        ZipObject.extract(
            Source_Name, path = mastr_directory
        )

!!! takes 15 minutes to compute !!!

In [58]:
mastr_directory = "/home/jan/Uni/DS-Project/data/MaStR/"
# xml to data frame
for i in range(len(Matches)-1):
    Source_Name = Matches[i][2:-2]
    Unit = mastr_directory + Source_Name
    print(Unit)
    # transform into python dictionary        
    tree = ET.parse(Unit)
    xml_data = tree.getroot()
    xmlstr = ET.tostring(xml_data, encoding='latin-1', method='xml')
    data_dict = dict(xmltodict.parse(xmlstr))
    # transform into json
    json_data = json.dumps(data_dict)
    with open(Source_Name[:-4] + ".json", "w") as json_file:
        json_file.write(json_data)

/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_1.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_10.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_11.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_12.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_13.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_14.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_15.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_16.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_17.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_18.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_19.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_2.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_20.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_21.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_22.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_23.xml
/home/jan/Uni/DS-Project/data/MaStR/EinheitenSolar_24.xml
/home/jan/Uni/DS